In [0]:
class CreateOrReplaceTables:
    def __init__(self,entity_name,ingestion_layer,truncate_flag,src_df):
        self.ingestion_layer = ingestion_layer
        self.entity_name = entity_name
        self.truncate_flag = truncate_flag
        self.src_df = src_df

    def generate_src_view(self):
        if self.ingestion_layer in ["bronze","silver"]:
            if self.src_df:
                pass
            else:
                if self.ingestion_layer == "bronze":
                    src_df= spark.read.parquet(f"{get_mnt('landing')}/{self.entity_name}/{self.entity_name}.parquet")
                else:
                    catalog_name = "bronze_incremental_schema" if self.entity_name.endswith('_incremental') else "bronze_schema"
                    src_df = spark.read.table(f"{catalog_name}.{self.entity_name.split('_inc')[0]}")
                self.src_df = src_df
    
    def create_or_replace_table(self,tgt_delta_path):
        catalog_name = self.ingestion_layer+"_incremental_schema" if self.entity_name.endswith('_incremental') else self.ingestion_layer+"_schema"
        table_name = self.entity_name.split('_inc')[0]
        src_view_name = f"{self.entity_name}_temp_view"
        self.src_df = self.src_df.withColumn("processing_dttm",current_timestamp())
        self.src_df.registerTempTable(src_view_name)
        col_names =','.join(self.src_df.columns)
        spark.sql(
                    f"""CREATE or REPLACE TABLE {catalog_name}.{table_name}
                USING DELTA
                LOCATION '{tgt_delta_path}'
                AS
                SELECT {col_names} FROM {src_view_name}
                """
                )
        if self.truncate_flag:
            spark.sql(f"truncate table {catalog_name}.{table_name}")
        spark.sql(f"DROP VIEW IF EXISTS {src_view_name}")
    
    def execute(self):
        self.generate_src_view()
        mnt_tgt = get_mnt(self.ingestion_layer)
        load_type = 'incremental_stg' if self.entity_name.endswith('_incremental') else 'stg'
        tgt_delta_path = f"dbfs:{mnt_tgt}/{self.entity_name.split('_')[0]}/{load_type}/{self.entity_name}_delta"
        self.create_or_replace_table(tgt_delta_path)

In [0]:
def crt_execute(entity_name,ingestion_layer,truncate_flag,src_df):
    CreateOrReplaceTables(entity_name,ingestion_layer,truncate_flag,src_df).execute()